In [1]:
import os 
import numpy as np
import pandas as pd
import datetime as dt 
import json 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import hmac
from geopy.distance import geodesic 
from sklearn.cluster import KMeans
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [2]:
df_all = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2020.csv")
df_all.head()

,{9FF0D969-77DC-11ED-E053-6C04A8C06383},190000,2020-02-06 00:00,BS15 3ER,T,N,F,4,Unnamed: 8,BRIARFIELD AVENUE,HANHAM,BRISTOL,CITY OF BRISTOL,CITY OF BRISTOL.1,A,A.1
0,{9FF0D969-77DE-11ED-E053-6C04A8C06383},244000,2020-01-31 00:00,BS5 8BL,T,N,F,113,NaN,TROOPERS HILL ROAD,NaN,BRISTOL,CITY OF BRISTOL,CITY OF BRISTOL,A,A
1,{9FF0D969-77DF-11ED-E053-6C04A8C06383},234000,2020-02-11 00:00,BS5 9EX,S,N,F,4,NaN,TRELAWNEY AVENUE,NaN,BRISTOL,CITY OF BRISTOL,CITY OF BRISTOL,A,A
2,{9FF0D969-77E1-11ED-E053-6C04A8C06383},206500,2020-02-07 00:00,BS5 8FB,F,Y,L,THE BOOT FACTORY,FLAT 8,BEACONSFIELD ROAD,ST GEORGE,BRISTOL,CITY OF BRISTOL,CITY OF BRISTOL,A,A
3,{9FF0D969-77E3-11ED-E053-6C04A8C06383},686000,2020-02-14 00:00,BS6 5UF,T,N,F,36,NaN,KINGSDOWN PARADE,NaN,BRISTOL,CITY OF BRISTOL,CITY OF BRISTOL,A,A
4,{9FF0D969-77E4-11ED-E053-6C04A8C06383},251500,2020-01-30 00:00,BS15 1BX,S,N,F,27,NaN,BROCKHURST GARDENS,NaN,BRISTOL,CITY OF BRISTOL,CITY OF BRISTOL,A,A


In [3]:
df_all.shape

(137230, 16)

In [4]:
df_all.columns = ['TUID', 'Price', 'Date_Transfer', 'Postalcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']
df_all['Date_Transfer'] = df_all['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2010
df_all.drop(df_all[df_all.Date_Transfer.dt.year < 2010].index, inplace=True)

df_all.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)
df_all_london = df_all.query("Town_City == 'LONDON'")
streets = df_all_london['Street'].unique().tolist()
df_grp_price = df_all_london.groupby(['Street'])['Price'].mean().reset_index()
df_grp_price.columns = ['Street', 'Avg_Price']

df_affordable = df_grp_price.query("(Avg_Price >= 2000000) & (Avg_Price <= 3000000)")
df_affordable

,Street,Avg_Price
35,ADDISON GARDENS,2766000.0
45,ADMIRALS WAY,2050000.0
57,AIREDALE AVENUE,2300000.0
64,ALBANY,3000000.0
80,ALDERNEY STREET,2260000.0
...,...,...
5071,WESTMORELAND ROAD,2700000.0
5121,WILDS RENTS,2100000.0
5128,WILLOUGHBY STREET,2010000.0
5130,WILMINGTON SQUARE,2417500.0


In [5]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 35
item: Street       ADDISON GARDENS
Avg_Price          2.766e+06
Name: 35, dtype: object
item.Street only: ADDISON GARDENS
index: 45
item: Street       ADMIRALS WAY
Avg_Price        2.05e+06
Name: 45, dtype: object
item.Street only: ADMIRALS WAY
index: 57
item: Street       AIREDALE AVENUE
Avg_Price            2.3e+06
Name: 57, dtype: object
item.Street only: AIREDALE AVENUE
index: 64
item: Street       ALBANY
Avg_Price     3e+06
Name: 64, dtype: object
item.Street only: ALBANY
index: 80
item: Street       ALDERNEY STREET
Avg_Price           2.26e+06
Name: 80, dtype: object
item.Street only: ALDERNEY STREET
index: 103
item: Street       ALLEYN PARK
Avg_Price        2.4e+06
Name: 103, dtype: object
item.Street only: ALLEYN PARK
index: 142
item: Street       AMWELL STREET
Avg_Price            3e+06
Name: 142, dtype: object
item.Street only: AMWELL STREET
index: 326
item: Street       BASILICA MEWS
Avg_Price          2.2e+06
Name: 326, dtype: object
item.Street only: BASILICA MEW

In [6]:
geolocator = Nominatim(user_agent="London_explorer")

In [7]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
df_affordable

,Street,Avg_Price,city_coord
35,ADDISON GARDENS,2766000.0,"(53.7642539, -0.3085823)"
45,ADMIRALS WAY,2050000.0,"(52.600467, 1.187511)"
57,AIREDALE AVENUE,2300000.0,"(53.4363678, -1.106305)"
64,ALBANY,3000000.0,"(42.6511674, -73.754968)"
80,ALDERNEY STREET,2260000.0,"(52.9458857, -1.170078)"
...,...,...,...
5071,WESTMORELAND ROAD,2700000.0,"(51.3936392, 0.0047098)"
5121,WILDS RENTS,2100000.0,"(51.4975037, -0.0848711)"
5128,WILLOUGHBY STREET,2010000.0,"(52.9277785, -1.212814)"
5130,WILMINGTON SQUARE,2417500.0,"(51.5267654, -0.11048536340380957)"


In [9]:

df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)
df_affordable

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Street,Avg_Price,city_coord,Latitude,Longitude
35,ADDISON GARDENS,2766000.0,"(53.7642539, -0.3085823)",53.764254,-0.308582
45,ADMIRALS WAY,2050000.0,"(52.600467, 1.187511)",52.600467,1.187511
57,AIREDALE AVENUE,2300000.0,"(53.4363678, -1.106305)",53.436368,-1.106305
64,ALBANY,3000000.0,"(42.6511674, -73.754968)",42.651167,-73.754968
80,ALDERNEY STREET,2260000.0,"(52.9458857, -1.170078)",52.945886,-1.170078
...,...,...,...,...,...
5071,WESTMORELAND ROAD,2700000.0,"(51.3936392, 0.0047098)",51.393639,0.004710
5121,WILDS RENTS,2100000.0,"(51.4975037, -0.0848711)",51.497504,-0.084871
5128,WILLOUGHBY STREET,2010000.0,"(52.9277785, -1.212814)",52.927779,-1.212814
5130,WILMINGTON SQUARE,2417500.0,"(51.5267654, -0.11048536340380957)",51.526765,-0.110485


In [10]:
df = df_affordable.drop(columns=['city_coord'])
df

,Street,Avg_Price,Latitude,Longitude
35,ADDISON GARDENS,2766000.0,53.764254,-0.308582
45,ADMIRALS WAY,2050000.0,52.600467,1.187511
57,AIREDALE AVENUE,2300000.0,53.436368,-1.106305
64,ALBANY,3000000.0,42.651167,-73.754968
80,ALDERNEY STREET,2260000.0,52.945886,-1.170078
...,...,...,...,...
5071,WESTMORELAND ROAD,2700000.0,51.393639,0.004710
5121,WILDS RENTS,2100000.0,51.497504,-0.084871
5128,WILLOUGHBY STREET,2010000.0,52.927779,-1.212814
5130,WILMINGTON SQUARE,2417500.0,51.526765,-0.110485


In [11]:
address = 'London, UK'

geolocator = Nominatim(user_agent='London_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [12]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [13]:
CLIENT_ID = 'JPYF4XBBRRCEP4FOBF5SCHGCIA4DFYDNCZYHUSN3PP51BSN4' 
CLIENT_SECRET = 'PQHXES2PX2LB5VWJ1RTOOQB0ZVMSBOQ4EA21C5VXBF134UB2' 
VERSION = '20200714' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JPYF4XBBRRCEP4FOBF5SCHGCIA4DFYDNCZYHUSN3PP51BSN4
CLIENT_SECRET:PQHXES2PX2LB5VWJ1RTOOQB0ZVMSBOQ4EA21C5VXBF134UB2


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
 
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ADDISON GARDENS
ADMIRALS WAY
AIREDALE AVENUE
ALBANY
ALDERNEY STREET
ALLEYN PARK
AMWELL STREET
BASILICA MEWS
BERKELEY PLACE
BLAKE GARDENS
BOURDON STREET
BRUTON STREET
BURNSALL STREET
CADOGAN STREET
CALLCOTT STREET
CAMPDEN HILL GATE
CAMPION ROAD
CANNON HILL
CARLISLE PLACE
CARTERET STREET
CASTELNAU
CENACLE CLOSE
CHANTRY SQUARE
CHAPLIN ROAD
CHELSEA EMBANKMENT
CHIDDINGSTONE STREET
CHILTERN STREET
CHIPSTEAD STREET
CHOLMELEY CRESCENT
CHRISTINA STREET
CHURTON STREET
CLARENDON ROAD
CLAREVILLE STREET
COLEHERNE MEWS
COLVILLE PLACE
CONDUIT MEWS
COTMAN CLOSE
CRISTOWE ROAD
CRONDACE ROAD
DENEWOOD ROAD
DRAX AVENUE
DRAYTON GARDENS
DUNDONALD ROAD
EAST DULWICH ROAD
EATON ROW
ECCLESTON MEWS
ESMOND ROAD
EVELYN GARDENS
FALCON ROAD
FARRIER WALK
FIRECREST DRIVE
FITZALAN ROAD
GEORGE STREET
GLOUCESTER CIRCUS
GLOUCESTER ROAD
GRANGE PARK
HALSEY STREET
HANS ROAD
HARTINGTON ROAD
HESTER ROAD
HIGH STREET NORTH
HIGHLEVER ROAD
HOLLAND PARK
HOLLAND PARK MEWS
HOLLAND STREET
HOLLYWOOD ROAD
HOME PARK ROAD
HOXTON SQUARE
HYD

In [16]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ADDISON GARDENS,53.764254,-0.308582,East Park,53.764543,-0.302553,Park
1,ADDISON GARDENS,53.764254,-0.308582,Co-op Food,53.766355,-0.311073,Grocery Store
2,ADDISON GARDENS,53.764254,-0.308582,Nisa Local,53.762906,-0.307124,Convenience Store
3,ADDISON GARDENS,53.764254,-0.308582,Pavilion Cafe,53.764220,-0.302551,Café
4,ADMIRALS WAY,52.600467,1.187511,Flowerdays,52.602280,1.184685,Tea Room
...,...,...,...,...,...,...,...
3547,WOODSFORD SQUARE,51.504037,-0.209483,Blagclub,51.500275,-0.210933,Nightclub
3548,WOODSFORD SQUARE,51.504037,-0.209483,Holland Park London Underground Station,51.507261,-0.205511,Metro Station
3549,WOODSFORD SQUARE,51.504037,-0.209483,TfL Santander Cycle Hire,51.500096,-0.211412,Bike Rental / Bike Share
3550,WOODSFORD SQUARE,51.504037,-0.209483,Watertower,51.504809,-0.216039,Scenic Lookout


In [17]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ADDISON GARDENS,4,4,4,4,4,4
ADMIRALS WAY,2,2,2,2,2,2
AIREDALE AVENUE,5,5,5,5,5,5
ALBANY,40,40,40,40,40,40
ALDERNEY STREET,15,15,15,15,15,15
...,...,...,...,...,...,...
WESTMORELAND ROAD,4,4,4,4,4,4
WILDS RENTS,49,49,49,49,49,49
WILLOUGHBY STREET,31,31,31,31,31,31


In [18]:
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 316 uniques categories.


In [19]:
location_venues.shape

(3552, 7)

In [20]:
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

venues_onehot['Street'] = location_venues['Street'] 
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ADDISON GARDENS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ADDISON GARDENS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ADDISON GARDENS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ADDISON GARDENS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ADMIRALS WAY,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ADDISON GARDENS,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,ADMIRALS WAY,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,AIREDALE AVENUE,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,ALBANY,0.0,0.0,0.000000,0.05,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,ALDERNEY STREET,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,WESTMORELAND ROAD,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.250000,0.0,0.0,0.0
124,WILDS RENTS,0.0,0.0,0.000000,0.00,0.020408,0.0,0.0,0.0,0.040816,...,0.0,0.020408,0.0,0.0,0.020408,0.0,0.020408,0.0,0.0,0.0
125,WILLOUGHBY STREET,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
126,WILMINGTON SQUARE,0.0,0.0,0.017241,0.00,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.017241,0.0,0.0,0.000000,0.0,0.017241,0.0,0.0,0.0


In [22]:
num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ADDISON GARDENS----
               venue  freq
0  Convenience Store  0.25
1      Grocery Store  0.25
2               Café  0.25
3               Park  0.25
4  Accessories Store  0.00


----ADMIRALS WAY----
               venue  freq
0                Pub   0.5
1           Tea Room   0.5
2  Accessories Store   0.0
3        Opera House   0.0
4       Outlet Store   0.0


----AIREDALE AVENUE----
                        venue  freq
0                         Pub   0.4
1  Construction & Landscaping   0.2
2                 Bridal Shop   0.2
3          Italian Restaurant   0.2
4         Japanese Restaurant   0.0


----ALBANY----
         venue  freq
0         Café  0.10
1          Pub  0.08
2        Hotel  0.08
3  Coffee Shop  0.08
4         Bank  0.05


----ALDERNEY STREET----
            venue  freq
0             Pub  0.13
1           Hotel  0.13
2  Sandwich Place  0.07
3    Optical Shop  0.07
4             Bar  0.07


----ALLEYN PARK----
                  venue  freq
0                   Pu

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [25]:
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [26]:
london_grouped=df

In [27]:

kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([4, 3, 0, 1, 0, 2, 1, 0, 1, 2, 4, 4, 0, 4, 3, 1, 0, 3, 0, 3, 2, 1,
       2, 4, 2, 4, 4, 4, 0, 0, 3, 0, 0, 0, 2, 1, 2, 2, 0, 2, 2, 1, 2, 0,
       4, 0, 3, 0, 2, 4], dtype=int32)

In [28]:
london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
35,ADDISON GARDENS,2766000.0,53.764254,-0.308582
45,ADMIRALS WAY,2050000.0,52.600467,1.187511
57,AIREDALE AVENUE,2300000.0,53.436368,-1.106305
64,ALBANY,3000000.0,42.651167,-73.754968
80,ALDERNEY STREET,2260000.0,52.945886,-1.170078


In [29]:
london_grouped_clustering['Cluster Labels'] = kmeans.labels_
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) 

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,ADDISON GARDENS,2766000.0,53.764254,-0.308582,4,Café,Convenience Store,Park,Grocery Store,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market
45,ADMIRALS WAY,2050000.0,52.600467,1.187511,3,Pub,Tea Room,Zoo,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
57,AIREDALE AVENUE,2300000.0,53.436368,-1.106305,0,Pub,Bridal Shop,Construction & Landscaping,Italian Restaurant,Zoo,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farmers Market
64,ALBANY,3000000.0,42.651167,-73.754968,1,Café,Hotel,Coffee Shop,Pub,Pizza Place,Bank,Park,American Restaurant,Food Truck,Mobile Phone Shop
80,ALDERNEY STREET,2260000.0,52.945886,-1.170078,0,Hotel,Pub,Coffee Shop,Fast Food Restaurant,Bar,Gym,Sandwich Place,Optical Shop,Auto Garage,Grocery Store
103,ALLEYN PARK,2400000.0,51.492742,-0.371967,2,Pub,Playground,Lawyer,Zoo,Fast Food Restaurant,English Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
142,AMWELL STREET,3000000.0,51.764879,-0.011473,1,Shopping Mall,Supermarket,Grocery Store,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Bakery,Event Space,Exhibit,Factory
326,BASILICA MEWS,2200000.0,40.748742,14.484494,0,Historic Site,History Museum,Italian Restaurant,Pizza Place,Diner,Campground,Campanian Restaurant,Public Art,Train Station,Arts & Crafts Store
414,BERKELEY PLACE,2900000.0,39.449940,-77.959741,1,Baseball Field,Gym,Zoo,French Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
475,BLAKE GARDENS,2500000.0,22.284280,114.148183,2,Japanese Restaurant,Café,Bar,Coffee Shop,Cocktail Bar,French Restaurant,Ramen Restaurant,Italian Restaurant,Thai Restaurant,Restaurant


In [30]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,2300000.0,Pub,Bridal Shop,Construction & Landscaping,Italian Restaurant,Zoo,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farmers Market
80,2260000.0,Hotel,Pub,Coffee Shop,Fast Food Restaurant,Bar,Gym,Sandwich Place,Optical Shop,Auto Garage,Grocery Store
326,2200000.0,Historic Site,History Museum,Italian Restaurant,Pizza Place,Diner,Campground,Campanian Restaurant,Public Art,Train Station,Arts & Crafts Store
731,2190000.0,Coffee Shop,Clothing Store,Pharmacy,Fast Food Restaurant,Bookstore,Hardware Store,Asian Restaurant,Toy / Game Store,Kids Store,Furniture / Home Store
804,2200000.0,Pub,Trail,Food,Zoo,Fast Food Restaurant,English Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant


In [32]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,3000000.0,Café,Hotel,Coffee Shop,Pub,Pizza Place,Bank,Park,American Restaurant,Food Truck,Mobile Phone Shop
142,3000000.0,Shopping Mall,Supermarket,Grocery Store,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Bakery,Event Space,Exhibit,Factory
414,2900000.0,Baseball Field,Gym,Zoo,French Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
801,3000000.0,Café,Garden,Clothing Store,Burger Joint,Italian Restaurant,Sporting Goods Shop,Coffee Shop,Park,Pub,Restaurant
903,3000000.0,Park,Coffee Shop,Breakfast Spot,Grocery Store,Food Truck,Health & Beauty Service,Zoo,Cycle Studio,Fish & Chips Shop,Exhibit


In [33]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
103,2400000.0,Pub,Playground,Lawyer,Zoo,Fast Food Restaurant,English Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
475,2500000.0,Japanese Restaurant,Café,Bar,Coffee Shop,Cocktail Bar,French Restaurant,Ramen Restaurant,Italian Restaurant,Thai Restaurant,Restaurant
870,2370000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
928,2550000.0,Hotel,Pub,Italian Restaurant,Restaurant,Café,Mediterranean Restaurant,French Restaurant,Juice Bar,Grocery Store,Chinese Restaurant
967,2500000.0,Pub,Café,Italian Restaurant,Plaza,Garden,English Restaurant,French Restaurant,Nightclub,Bakery,Gym / Fitness Center


In [34]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,2050000.0,Pub,Tea Room,Zoo,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
772,2125000.0,Pub,Park,Coffee Shop,Ice Cream Shop,Bakery,Yoga Studio,Indian Restaurant,Grocery Store,Hotel,Food & Drink Shop
817,2000000.0,Beach,Zoo,Fried Chicken Joint,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
863,2000000.0,Donut Shop,Burger Joint,Gas Station,Bakery,Student Center,Deli / Bodega,Hookah Bar,Café,Pub,Flea Market
1050,2000000.0,Café,Coffee Shop,Bar,Park,Pizza Place,Hotel,Restaurant,Japanese Restaurant,Ice Cream Shop,Farmers Market


In [35]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,2766000.0,Café,Convenience Store,Park,Grocery Store,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market
530,2600000.0,Sandwich Place,Pub,Park,Fish & Chips Shop,Performing Arts Venue,Falafel Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space
692,2675000.0,Coffee Shop,American Restaurant,Ice Cream Shop,Discount Store,Fish Market,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
762,2763050.0,Convenience Store,Department Store,Discount Store,Outlet Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
933,2647000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
